# Detection of synchronization artifact in EEG recordings

In [ ]:
import sys
import os

# Get the path to the project root (one level up from the notebook)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root, NOT 'source', to sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# print the sys.path
print(sys.path)

# IMPORTANT: Set Qt backend for GUI functionality BEFORE importing matplotlib.pyplot
import matplotlib
matplotlib.use('QtAgg')  # Required for manual_select_sync GUI
print(f"Matplotlib backend: {matplotlib.get_backend()}")

import numpy as np
import pandas as pd
import mne
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import plotly.graph_objects as go

from dbs_eeg_sync.sync_artifact_finder import detect_eeg_sync_artifact


In [ ]:
# run for all patients
sub_ids = [ 'P4-2001', 'P4-2002', 'P4-2003', 'P4-2004', 'P4-2005', 'P4-2007', 'P4-2008', 'P4-2009', 'P4-2010', 'P4-2011', 'P4-2012']
# sub_id = 'P4-2001'
block = 'baseline'
dataDir_server = r"/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB"


# loop through sub_ids
for sub_id in sub_ids:
    eeg_file = os.path.join(dataDir_server, sub_id, 'RawData', 'EEG', block, block + '_raw.set')
    eeg_data = mne.io.read_raw_eeglab(eeg_file, preload=True)
    if sub_id == 'P4-2007':
        freq_low, freq_high = 110, 120
    else:
        freq_low, freq_high = 120, 130

    channel, eeg_sync_idx, eeg_sync_s, result, smoothed_power = detect_eeg_sync_artifact(
        eeg_data, 
        freq_low=freq_low, 
        freq_high=freq_high, 
        time_range=(0, 120), 
        plot=True, 
        save_dir=None, 
        sub_id=sub_id, 
        block=block
    )

## Manual Sync Selection with GUI

**Note:** The GUI requires the QtAgg backend (set in Cell 1). If you get a backend error:
1. Restart the kernel
2. Re-run Cell 1 which sets `matplotlib.use('QtAgg')`
3. Then run this cell


In [ ]:
# Import the manual sync GUI
from dbs_eeg_sync.gui import manual_select_sync

sub_id = 'P4-2010'
block = 'baseline'
freq_low, freq_high = 120, 130

eeg_file = os.path.join(dataDir_server, sub_id, 'RawData', 'EEG', block, block + '_raw.set')
eeg_data = mne.io.read_raw_eeglab(eeg_file, preload=True)

# Pass raw EEG data and let GUI compute the power
# The GUI will automatically compute power for the specified frequency band
# NOTE: QtAgg backend was set in Cell 1 - required for GUI to work in notebooks
eeg_idx, eeg_time, dbs_idx, dbs_time = manual_select_sync(
    eeg_data=eeg_data,
    eeg_fs=eeg_data.info['sfreq'],
    dbs_data=None,
    dbs_fs=None,
    title=f"{sub_id} - {block}",
    freq_low=freq_low,
    freq_high=freq_high,
    channel='POz'  # Will use this channel for power computation
)

print(f"Selected EEG sync: index={eeg_idx}, time={eeg_time:.3f}s")
print(f"DBS sync: {dbs_idx}, {dbs_time} (not used in this example)")

### Example: Manual sync with both EEG and DBS data


In [ ]:
# Example with both EEG and DBS data
# Uncomment to run this example

# from dbs_eeg_sync.data_loader import open_json_file, read_time_domain_data

# # Load EEG
# sub_id = 'P4-2004'
# block = 'baseline'
# eeg_file = os.path.join(dataDir_server, sub_id, 'RawData', 'EEG', block, block + '_raw.set')
# eeg_data = mne.io.read_raw_eeglab(eeg_file, preload=True)

# # Load DBS
# file_info_dbs = [
#     {"sub_id": "P4-2004", "file_path": "/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB/P4-2004/RawData/DBS/Report_Json_Session_Report_20241028T163753.json"},
# ]
# block_number = 0 if block == 'baseline' else 0
# dbs_path = [file for file in file_info_dbs if file['sub_id'] == sub_id][block_number]['file_path']
# json_data = open_json_file(dbs_path)
# dbs_data = read_time_domain_data(json_data, block_number) 
# dbs_signal = dbs_data["TimeDomainData"].values
# dbs_fs = dbs_data["SampleRateInHz"][0]

# # Call manual sync with both signals
# eeg_idx, eeg_time, dbs_idx, dbs_time = manual_select_sync(
#     eeg_data=eeg_data,
#     eeg_fs=eeg_data.info['sfreq'],
#     dbs_data=dbs_signal,
#     dbs_fs=dbs_fs,
#     title=f"{sub_id} - {block} - Manual Sync",
#     freq_low=120,
#     freq_high=130,
#     channel='POz'
# )

# print(f"Selected EEG sync: index={eeg_idx}, time={eeg_time:.3f}s")
# print(f"Selected DBS sync: index={dbs_idx}, time={dbs_time:.3f}s")


### Alternative: Using pre-computed power data


In [ ]:
# If you already have computed power and want to use that directly:
# Uncomment to run this example

# from dbs_eeg_sync.power_calculator import compute_samplewise_eeg_power

# # Load EEG and compute power first
# sub_id = 'P4-2010'
# block = 'baseline'
# eeg_file = os.path.join(dataDir_server, sub_id, 'RawData', 'EEG', block, block + '_raw.set')
# eeg_data = mne.io.read_raw_eeglab(eeg_file, preload=True)

# # Pre-compute power
# power_signal, power_times = compute_samplewise_eeg_power(
#     eeg_data, 
#     freq_low=120, 
#     freq_high=130, 
#     channel='POz'
# )

# # Pass the power signal directly (no freq_low, freq_high, channel needed)
# eeg_idx, eeg_time, dbs_idx, dbs_time = manual_select_sync(
#     eeg_data=power_signal,  # Pass power array directly
#     eeg_fs=eeg_data.info['sfreq'],
#     dbs_data=None,
#     dbs_fs=None,
#     title=f"{sub_id} - {block} - Pre-computed Power"
# )

# print(f"Selected EEG sync: index={eeg_idx}, time={eeg_time:.3f}s")


# Investigate the faulty trials

In [ ]:
sub_id = 'P4-2010'
block = 'baseline'
dataDir_server = r"/Volumes/03_Neurofeedback/project_only/02_Data/Patient_NFB"


# loop through sub_ids
eeg_file = os.path.join(dataDir_server, sub_id, 'RawData', 'EEG', block, block + '_raw.set')
eeg_data = mne.io.read_raw_eeglab(eeg_file, preload=True)
freq_low, freq_high = 120, 130
if sub_id == 'P4-2007':
    freq_low, freq_high = 110, 120

print(eeg_data.ch_names)

channel, eeg_sync_idx, eeg_sync_s, result, smoothed_power = detect_eeg_sync_artifact(
    eeg_data, 
    freq_low=freq_low, 
    freq_high=freq_high, 
    plot=True, 
    save_dir=None, 
    sub_id=sub_id, 
    block=block
)